In [1]:
# The code was removed by DSX for sharing.

In [2]:
# The code was removed by DSX for sharing.

In [1]:
import gc

#from kaggler.online_model import FTRL
from sklearn.feature_extraction import FeatureHasher
from scipy import sparse
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.utils import resample
from sklearn import metrics
import pandas as pd
from ggplot import *
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
matplotlib.style.use('ggplot')

/gpfs/fs01/user/sebe-fe5f7e5ad401e7-5fb1bc63e22e/.local/lib/python2.7/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/gpfs/fs01/user/sebe-fe5f7e5ad401e7-5fb1bc63e22e/.local/lib/python2.7/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/gpfs/fs01/user/sebe-fe5f7e5ad401e7-5fb1bc63e22e/.local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
train = pd.read_csv(get_object_storage_file_with_credentials_501d5c35b97c40ce8bc204fdd04461d2('MAchine', 'train.csv'))
test = pd.read_csv(get_object_storage_file_with_credentials_501d5c35b97c40ce8bc204fdd04461d2('MAchine', 'test.csv'))

In [5]:
print train.shape
print test.shape

(12137810, 10)
(3706907, 9)


In [ ]:
trainy=train.loc[:,'click']
train.drop(['click','ID'],axis=1,inplace=True)
train_x,test_x,train_y,test_y=train_test_split(train,trainy,test_size=0.33,random_state=17)

<h3>Analysing train and test data</h3>

In [6]:
test_id = test['ID']
train.drop(['ID'],axis=1,inplace=True)
test.drop(['ID'],axis=1,inplace=True)

In [7]:
print train.shape
print test.shape

(12137810, 9)
(3706907, 8)


In [8]:
target =train['click']
train.drop(['click'],axis=1,inplace=True)

In [9]:
combine =  train.append(test)

In [10]:
print combine.shape

(15844717, 8)


In [11]:
print combine.isnull().sum()

datetime             0
siteid         1583291
offerid              0
category             0
merchant             0
countrycode          0
browserid       830233
devid          2524918
dtype: int64


In [12]:
combine.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
siteid,14261426.0,5.029348e+06,2.891212e+06,71.0,2539650.0,4997276.0,7558174.0,9999996.0
offerid,15844717.0,4.940615e+05,2.909011e+05,0.0,242089.0,490933.0,746706.0,999998.0
category,15844717.0,4.186201e+04,2.923686e+04,97.0,14960.0,40339.0,64452.0,99914.0
merchant,15844717.0,4.893830e+07,2.879568e+07,260443.0,26391770.0,47079934.0,74363610.0,99770433.0


Filling missing values

In [13]:
l=['Chrome', 'Edge', 'Firefox', 'Google Chrome', 'IE',
       'Internet Explorer', 'InternetExplorer', 'Mozilla',
       'Mozilla Firefox', 'Opera', 'Safari']
for i in l:
    print i + str(np.unique(combine[combine.browserid == i]['devid']))
    combine.loc[:,'devid'][(
        combine.loc[:,'browserid'] == i)&(combine.loc[:,'devid'].isnull())] = np.unique(combine.loc[:,'devid'][combine.loc[:,'browserid'] == i])[1]
print 'Imputation Done'     

Chrome[nan nan nan ..., nan nan 'Desktop']
Edge[nan nan nan ..., nan nan 'Tablet']
Firefox[nan nan nan ..., nan nan 'Mobile']
Google Chrome[nan nan nan ..., nan nan 'Mobile']
IE[nan nan nan ..., nan nan 'Mobile']
Internet Explorer[nan nan nan ..., nan nan 'Tablet']
InternetExplorer[nan nan nan ..., nan nan 'Desktop']
Mozilla[nan nan nan ..., nan nan 'Desktop']
Mozilla Firefox[nan nan nan ..., nan nan 'Desktop']
Opera[nan nan nan ..., nan nan 'Mobile']
Safari[nan nan nan ..., nan nan 'Tablet']
Imputation Done


In [14]:
print combine.isnull().sum()

datetime             0
siteid         1583291
offerid              0
category             0
merchant             0
countrycode          0
browserid       830233
devid          2524918
dtype: int64


In [15]:
combine.browserid[(combine.browserid.isnull())& (combine['devid'] == 'Desktop')]='Mozilla'
combine.browserid[(combine.browserid.isnull())& (combine['devid'] == 'Mobile')]='Opera'
combine.browserid[(combine.browserid.isnull())& (combine['devid'] == 'Tablet')]='Edge'
combine.loc[:,'browserid'][(combine.loc[:,'browserid'].isnull()) & (combine.loc[:,'devid'].isnull())]='Other'
combine.loc[:,'devid'][(combine.loc[:,'devid'].isnull())]='Other'

In [16]:
combine.siteid.fillna(5029348,inplace=True)

In [17]:
combine=combine.assign(mercat=combine.category*combine.merchant)
combine=combine.assign(offcat=combine.offerid*combine.category)
combine=combine.assign(meroff=combine.offerid*combine.merchant)

In [19]:
combine['datetime'] = pd.to_datetime(combine['datetime'])
combine['tweekday'] = combine['datetime'].dt.weekday
combine['thour'] = combine['datetime'].dt.hour
combine['tminute'] = combine['datetime'].dt.minute

In [20]:
combine.drop(['datetime'],axis=1,inplace=True)

In [ ]:
del train;
del test;
del train_x,test_x,train_y,test_y;
gc.collect()

In [22]:
newtrain = pd.get_dummies(combine[0:12137810])

In [23]:
newtest= pd.get_dummies(combine[12137810:])

In [24]:
print newtrain.shape
print newtest.shape

(12137810, 32)
(3706907, 32)


In [25]:
train_x,test_x,train_y,test_y=train_test_split(newtrain[:len(target)],target,test_size=0.33,random_state=17)

In [98]:
clf = FTRL(a=.05, # alpha in the per-coordinate rate
b=1., # beta in the per-coordinate rate
l1=1.5, # L1 regularization parameter
l2=1.5, # L2 regularization parameter
n=2**20, # number of hashed features
epoch=5, # number of epochs
interaction=True) 

In [55]:
xtrains=sparse.csr_matrix(train_x)
xtests=sparse.csr_matrix(test_x)
stest=sparse.csr_matrix(newtest)

In [81]:
strains=sparse.csr_matrix(newtrain)

In [ ]:
clf.fit(strains,target.values)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(test_y, preds)
auc = metrics.auc(fpr,tpr)
print 'AUC Score:'+str(auc)
df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
ggplot(df, aes(x='fpr', y='tpr')) +geom_line() + geom_abline(linetype='dashed')

In [ ]:
click=clf.predict(stest)

In [ ]:
final =pd.DataFrame(test_id)
sol = pd.concat([final,pd.DataFrame(click)],axis=1)
sol.columns =['ID','click']

In [ ]:
sol.to_csv('FTRL6.csv',quoting=False,index=False)

In [ ]:
put_file(credentials_503,'FTRL6.csv')